# Project to predict Titanic survivors

In [325]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import math
train_data=open('titanic_x_y_train.csv')
train_data=pd.read_csv(train_data)
test_data=open('titanic_x_test.csv')
test_data=pd.read_csv(test_data)

In [296]:
train_data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,2,"Weisz, Mrs. Leopold (Mathilde Francoise Pede)",female,29.0,1,0,228414,26.000,NaN,S,1
1,3,"Williams, Mr. Howard Hugh ""Harry""",male,NaN,0,0,A/5 2466,8.050,NaN,S,0
2,2,"Morley, Mr. Henry Samuel (""Mr Henry Marshall"")",male,39.0,0,0,250655,26.000,NaN,S,0
3,3,"Palsson, Mrs. Nils (Alma Cornelia Berglund)",female,29.0,0,4,349909,21.075,NaN,S,0
4,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.050,NaN,S,0


In [297]:
test_data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,"Davies, Master. John Morgan Jr",male,8.0,1,1,C.A. 33112,36.7500,NaN,S
1,1,"Leader, Dr. Alice (Farnham)",female,49.0,0,0,17465,25.9292,D17,S
2,3,"Kilgannon, Mr. Thomas J",male,NaN,0,0,36865,7.7375,NaN,Q
3,2,"Jacobsohn, Mrs. Sidney Samuel (Amy Frances Chr...",female,24.0,2,1,243847,27.0000,NaN,S
4,1,"McGough, Mr. James Robert",male,36.0,0,0,PC 17473,26.2875,E25,S


In [298]:
print(train_data.shape)
print(test_data.shape)

(668, 11)
(223, 10)


In [299]:
train_data=train_data.drop('Name',axis=1)
train_data=train_data.drop('Cabin',axis=1)
train_data=train_data.drop('Ticket',axis=1)
train_data=train_data.dropna()

y_train=train_data['Survived']
x_train=train_data.drop('Survived',axis=1)

#x_train=np.array(x_train)
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,2,female,29.0,1,0,26.0000,S
2,2,male,39.0,0,0,26.0000,S
3,3,female,29.0,0,4,21.0750,S
4,3,male,25.0,0,0,7.0500,S
5,3,male,34.5,0,0,6.4375,C


In [300]:
x_train = pd.get_dummies(x_train, columns=["Pclass","Sex", "Embarked"])
x_train.head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,29.0,1,0,26.0000,0,1,0,1,0,0,0,1
2,39.0,0,0,26.0000,0,1,0,0,1,0,0,1
3,29.0,0,4,21.0750,0,0,1,1,0,0,0,1
4,25.0,0,0,7.0500,0,0,1,0,1,0,0,1
5,34.5,0,0,6.4375,0,0,1,0,1,1,0,0


In [301]:
x_train =np.array(x_train)
x_train=preprocessing.scale(x_train)
x_train= pd.DataFrame(x_train)
x_train['c']=1
x_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,c
0,-0.048122,0.533229,-0.509394,-0.169449,-0.586704,1.696117,-0.972345,1.294218,-1.294218,-0.482409,-0.186591,0.534844,1
1,0.654336,-0.572497,-0.509394,-0.169449,-0.586704,1.696117,-0.972345,-0.772667,0.772667,-0.482409,-0.186591,0.534844,1
2,-0.048122,-0.572497,3.886182,-0.273830,-0.586704,-0.589582,1.028442,1.294218,-1.294218,-0.482409,-0.186591,0.534844,1
3,-0.329105,-0.572497,-0.509394,-0.571078,-0.586704,-0.589582,1.028442,-0.772667,0.772667,-0.482409,-0.186591,0.534844,1
4,0.338230,-0.572497,-0.509394,-0.584059,-0.586704,-0.589582,1.028442,-0.772667,0.772667,2.072928,-0.186591,-1.869705,1


In [302]:
x_train=np.array(x_train)
y_train=np.array(y_train)

In [303]:
def sigmoid(x):
        return 1 / (1 + np.exp(-x))


def single_gradient(x_train,y_train, learning_rate,m):  
    m_slope = [0 for i in range(x_train.shape[1])]
    M = x_train.shape[0]
    N = x_train.shape[1]
    for j in range(N):
        for i in range(M):
            x = x_train[i]
            y = y_train[i]
            a=np.dot(m,x)
            m_slope[j] += (-1/M)* (y - sigmoid(a))*x[j]
    m_slope=np.array(m_slope)
    new_m=m-m_slope*learning_rate   
    return new_m
            
def gd(x_train,y_train, learning_rate, num_iterations):
    m = [0 for i in range(x_train.shape[1])]
    for i in range(num_iterations):
        m = single_gradient(x_train,y_train,learning_rate,m)
        #print(i, " Cost: ", cost(x_train,y_train,m))
    return m

def cost(x_train,y_train, m):
    total_cost = 0
    M = x_train.shape[0]
    for i in range(M):
        x = x_train[i]
        y = y_train[i]
        a = np.dot(m,x)
        total_cost += (1/M)*(-y*math.log(sigmoid(a))-(1-y)*math.log(1-sigmoid(a)))
    return total_cost

def fit(x_train,y_train):
    learning_rate =4
    num_iterations = 16
    m = gd(x_train,y_train, learning_rate, num_iterations)
    return m

In [304]:
fit(x_train,y_train)

array([-0.61497294, -0.30549235, -0.11488862,  0.04057852,  0.57762532,
        0.04613303, -0.54480571,  0.63351605, -0.63351605,  0.06560666,
       -0.11867316, -0.01028189, -0.39041529])

In [305]:
def prob(x_test):
    m=fit(x_train,y_train)
    Prob=[]
    for i in range(len(x_test)):
        x=x_test[i]
        a=np.dot(m,x)
        p = sigmoid(a)
        Prob.append(p)
    return Prob

In [306]:
def predict(x_test):
    Prob=prob(x_test)
    y_predicted = [1 if i > 0.6 else 0 for i in Prob]
    return np.array(y_predicted)

In [307]:
y_predict=predict(x_train)
y_predict

array([1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,

In [308]:
def accuracy(x_test):
    a=y_predict-y_train
    count=0
    for i in a:
        if i==0:
            count+=1
    accuracy=count/len(a)
    return accuracy 

In [310]:
train_accuracy=accuracy(x_train)
train_accuracy

0.7981308411214953

**predict test dataset**

In [326]:
test_data=test_data.drop('Name',axis=1)
test_data=test_data.drop('Cabin',axis=1)
test_data=test_data.drop('Ticket',axis=1)
test_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,2,male,8.0,1,1,36.7500,S
1,1,female,49.0,0,0,25.9292,S
2,3,male,NaN,0,0,7.7375,Q
3,2,female,24.0,2,1,27.0000,S
4,1,male,36.0,0,0,26.2875,S


In [327]:
test_data["Age"] = test_data["Age"].fillna(value= test_data["Age"].mean())
test_data["Embarked"] = test_data["Embarked"].fillna(value= test_data["Embarked"].mode())
test_data["Fare"] = test_data["Fare"].fillna(value= test_data["Fare"].mean())

In [328]:
test_data = pd.get_dummies(test_data, columns=["Pclass","Sex", "Embarked"])
test_data.head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,8.000000,1,1,36.7500,0,1,0,0,1,0,0,1
1,49.000000,0,0,25.9292,1,0,0,1,0,0,0,1
2,29.694775,0,0,7.7375,0,0,1,0,1,0,1,0
3,24.000000,2,1,27.0000,0,1,0,1,0,0,0,1
4,36.000000,0,0,26.2875,1,0,0,0,1,0,0,1


In [329]:
test_data =np.array(test_data)
test_data=preprocessing.scale(test_data)
test_data= pd.DataFrame(test_data)
test_data['c']=1
test_data=np.array(test_data)

In [330]:
y_predict=predict(test_data)
y_predict

array([0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 0])

In [331]:
 np.savetxt('predictions5.csv',y_predict)